Set `ssd_path` to a folder in which a file can be placed. It does not need to be "ssd" drive, but faster drives are better.

In [3]:
import time
from pathlib import Path
import numpy as np
from bed_reader import create_bed, open_bed

ssd_path = Path("m:/deldir/bench2")

On my system, it takes about 5 minutes to generate the 61 GB file (1/16th of a 1TB drive). The file is filed with a pattern of 0, 1, and 2 values.

In this case, we are pretending that we can more easily get all the SNP information for one individual at a time, so we write the file in the (less common) "individual-major" format.

In [ ]:
iid_count = 250_000
sid_count = 1_000_000

file_path = ssd_path / f"{iid_count}x{sid_count}mode0.bed"

snp_row = np.array(range(sid_count)) % 3
snp_row = snp_row.astype(np.uint8)

start_time = time.time()
with create_bed(file_path, iid_count=iid_count, sid_count=sid_count, major="individual") as bed_writer:
    for iid_index in range(iid_count):
        if iid_index % 1_000 == 0:
            current_time = time.time()
            elapsed_time = current_time - start_time
            if iid_index > 0:
                estimated_total_time = elapsed_time / iid_index * iid_count
                time_remaining = estimated_total_time - elapsed_time
                print(f"iid_index={iid_index:_}, Time elapsed: {elapsed_time:.2f} s, Estimated total time: {estimated_total_time:.2f} s, Time remaining: {time_remaining:.2f} s")
            else:
                print(f"Starting processing at iid_index={iid_index:_}")
        bed_writer.write(snp_row)
        snp_row[iid_index] = (snp_row[iid_index] + 1) % 3
total_time = time.time() - start_time
print(f"Processing complete. Total time taken: {total_time:.2f} s")        


We read from the file to check that it contains the expected values.

In [7]:
with open_bed(file_path) as bed_reader:
    val = bed_reader.read(np.s_[:10, :10])

assert np.all(
    val
    == [
        [0.0, 1.0, 2.0, 0.0, 1.0, 2.0, 0.0, 1.0, 2.0, 0.0],
        [1.0, 1.0, 2.0, 0.0, 1.0, 2.0, 0.0, 1.0, 2.0, 0.0],
        [1.0, 2.0, 2.0, 0.0, 1.0, 2.0, 0.0, 1.0, 2.0, 0.0],
        [1.0, 2.0, 0.0, 0.0, 1.0, 2.0, 0.0, 1.0, 2.0, 0.0],
        [1.0, 2.0, 0.0, 1.0, 1.0, 2.0, 0.0, 1.0, 2.0, 0.0],
        [1.0, 2.0, 0.0, 1.0, 2.0, 2.0, 0.0, 1.0, 2.0, 0.0],
        [1.0, 2.0, 0.0, 1.0, 2.0, 0.0, 0.0, 1.0, 2.0, 0.0],
        [1.0, 2.0, 0.0, 1.0, 2.0, 0.0, 1.0, 1.0, 2.0, 0.0],
        [1.0, 2.0, 0.0, 1.0, 2.0, 0.0, 1.0, 2.0, 2.0, 0.0],
        [1.0, 2.0, 0.0, 1.0, 2.0, 0.0, 1.0, 2.0, 0.0, 0.0],
    ]
)

In [8]:
bed_reader = open_bed(file_path) # open and keep open
bed_reader.shape

(250000, 1000000)

Now we use `bed-reader` to convert the file the less common individual-major mode (also known as "mode 0") into the usual SNP-major mode (mode 1).

On my machine, this takes about an hour. The script works my reading the data for 1000 SNPs at a time. This requires more memory than reading data of 1 SNP at a time, but is much, much faster. If you need to use less memory, change `sid_at_a_time` to a smaller number such as 500, 250, 100, etc.

In [ ]:
sid_at_a_time = 1000
file_path1 = ssd_path / f"{iid_count}x{sid_count}mode1.bed"
start_time = time.time()

assert (
    bed_reader.major == "individual"
), "No need to transpose if major is already SNP-major"

with create_bed(
    file_path1,
    iid_count=bed_reader.iid_count,
    sid_count=bed_reader.sid_count,
    properties=bed_reader.properties,
    major="SNP",
) as bed_writer:
    for sid_index in range(0, bed_reader.sid_count, sid_at_a_time):
        if sid_index % 1 == 0:
            current_time = time.time()
            elapsed_time = current_time - start_time
            if sid_index > 0:
                estimated_total_time = elapsed_time / sid_index * sid_count
                time_remaining = estimated_total_time - elapsed_time
                print(
                    f"sid_index={sid_index:_}, Time elapsed: {elapsed_time:.2f} s, Estimated total time: {estimated_total_time:.2f} s, Time remaining: {time_remaining:.2f} s"
                )
            else:
                print(f"Starting processing at sid_index={sid_index:_}")
        iid_column_by_chunk = bed_reader.read(
            np.s_[:, sid_index : sid_index + sid_at_a_time], dtype=np.int8
        )
        for iid_column in iid_column_by_chunk.T:
            bed_writer.write(iid_column)
total_time = time.time() - start_time
print(f"Processing complete. Total time taken: {total_time:.2f} s")

We confirm that the new copy of the file is the same size and starts with the same values as before.

In [11]:
with open_bed(file_path1) as bed_reader1:
    print(bed_reader1.shape)
    val1 = bed_reader1.read(np.s_[:10, :10])
assert np.all(
    val1    == [
        [0.0, 1.0, 2.0, 0.0, 1.0, 2.0, 0.0, 1.0, 2.0, 0.0],
        [1.0, 1.0, 2.0, 0.0, 1.0, 2.0, 0.0, 1.0, 2.0, 0.0],
        [1.0, 2.0, 2.0, 0.0, 1.0, 2.0, 0.0, 1.0, 2.0, 0.0],
        [1.0, 2.0, 0.0, 0.0, 1.0, 2.0, 0.0, 1.0, 2.0, 0.0],
        [1.0, 2.0, 0.0, 1.0, 1.0, 2.0, 0.0, 1.0, 2.0, 0.0],
        [1.0, 2.0, 0.0, 1.0, 2.0, 2.0, 0.0, 1.0, 2.0, 0.0],
        [1.0, 2.0, 0.0, 1.0, 2.0, 0.0, 0.0, 1.0, 2.0, 0.0],
        [1.0, 2.0, 0.0, 1.0, 2.0, 0.0, 1.0, 1.0, 2.0, 0.0],
        [1.0, 2.0, 0.0, 1.0, 2.0, 0.0, 1.0, 2.0, 2.0, 0.0],
        [1.0, 2.0, 0.0, 1.0, 2.0, 0.0, 1.0, 2.0, 0.0, 0.0],
    ]
)

(250000, 1000000)
